# Ungraded Lab: Tubería de ingeniería de características


En este laboratorio, seguirá explorando [Tensorflow Transform](https://www.tensorflow.org/tfx/transform/get_started). Esta vez, será en el contexto de una tubería de aprendizaje automático (ML). En los proyectos de grado de producción, usted quiere agilizar las tareas para que pueda mejorar más fácilmente su modelo o encontrar los problemas que puedan surgir. [Tensorflow Extended (TFX)](https://www.tensorflow.org/tfx) proporciona componentes que trabajan juntos para ejecutar los pasos más comunes en un proyecto de aprendizaje automático. Si quieres profundizar en las motivaciones que hay detrás de TFX y en la necesidad de los pipelines de aprendizaje automático, puedes leer sobre ello en [este documento](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b500d77bc4f518a1165c0ab43c8fac5d2948bc14.pdf) y en este [blog post](https://blog.tensorflow.org/2020/09/brief-history-of-tensorflow-extended-tfx.html).

Construirás pipelines de principio a fin en futuros cursos, pero para este, sólo construirás la parte de ingeniería de características. Específicamente, usted:

* ingestarás datos de un directorio base con `ExampleGen`.
* calcularás las estadísticas de los datos de entrenamiento con `StatisticsGen`.
* inferir un esquema con `SchemaGen`.
* detectar anomalías en los datos de evaluación con `ExampleValidator`.
* preprocesar los datos en características adecuadas para el entrenamiento del modelo con `Transform`.

Si varios de los pasos mencionados anteriormente te suenan, es porque los componentes TFX que se encargan de la validación y el análisis de los datos (es decir, `StatisticsGen`, `SchemaGen`, `ExampleValidator`) utilizan [Tensorflow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) bajo el capó. Ya estás familiarizado con esta librería por los ejercicios de la semana 1 y esta semana verás cómo encaja en un pipeline de ML.

Los componentes que utilizarás son los recuadros naranjas resaltados en la siguiente figura:

<img src='img/feature_eng_pipeline.png'>


<details><summary>Texto Original</summary>

# Ungraded Lab: Feature Engineering Pipeline


In this lab, you will continue exploring [Tensorflow Transform](https://www.tensorflow.org/tfx/transform/get_started). This time, it will be in the context of a machine learning (ML) pipeline. In production-grade projects, you want to streamline tasks so you can more easily improve your model or find issues that may arise. [Tensorflow Extended (TFX)](https://www.tensorflow.org/tfx) provides components that work together to execute the most common steps in a machine learning project. If you want to dig deeper into the motivations behind TFX and the need for machine learning pipelines, you can read about it in [this paper](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b500d77bc4f518a1165c0ab43c8fac5d2948bc14.pdf) and in this [blog post](https://blog.tensorflow.org/2020/09/brief-history-of-tensorflow-extended-tfx.html).

You will build end-to-end pipelines in future courses but for this one, you will only build up to the feature engineering part. Specifically, you will:

* ingest data from a base directory with `ExampleGen`
* compute the statistics of the training data with `StatisticsGen`
* infer a schema with `SchemaGen`
* detect anomalies in the evaluation data with `ExampleValidator`
* preprocess the data into features suitable for model training with `Transform`

If several steps mentioned above sound familiar, it's because the TFX components that deal with data validation and analysis (i.e. `StatisticsGen`, `SchemaGen`, `ExampleValidator`) uses [Tensorflow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) under the hood. You're already familiar with this library from the exercises in Week 1 and for this week, you'll see how it fits within an ML pipeline.

The components you will use are the orange boxes highlighted in the figure below:

<img src='img/feature_eng_pipeline.png'>



### Setup
    
    
### Importar paquetes

Empecemos por importar los paquetes y módulos necesarios. En caso de que quieras replicar esto en tu estación de trabajo local, usamos *Tensorflow v2.6* y *TFX v1.3.0*.

<details><summary>Texto Original</summary>

## Setup
    
    
### Import packages

Let's begin by importing the required packages and modules. In case you want to replicate this in your local workstation, we used *Tensorflow v2.6* and *TFX v1.3.0*.

In [1]:
import tensorflow as tf

from tfx import v1 as tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

### Definir rutas

Definirá algunas variables globales para indicar las rutas en el espacio de trabajo local.

<details><summary>Texto Original</summary>


### Define paths

You will define a few global variables to indicate paths in the local workspace.

In [2]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'

# directory of the raw data files
_data_root = './data/census_data'

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'adult.data')

### Vista previa del dataset

Volverá a utilizar el conjunto de datos [Census Income dataset](https://archive.ics.uci.edu/ml/datasets/Adult) del laboratorio no calificado de la semana 1 para que pueda comparar los resultados cuando se utiliza solo el TFDV y cuando se utiliza bajo el TFX. Sólo para recordar, los datos pueden ser utilizados para predecir si un individuo gana más o menos de 50 mil dólares anuales. Aquí está la descripción de las características de nuevo: 

* **age**: continuous.
* **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* **fnlwgt**: continuous.
* **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* **education-num**: continuous.
* **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* **race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* **sex**: Female, Male.
* **capital-gain**: continuous.
* **capital-loss**: continuous.
* **hours-per-week**: continuous.
* **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

<details><summary>Texto Original</summary>


### Preview the  dataset

You will again be using the [Census Income dataset](https://archive.ics.uci.edu/ml/datasets/Adult) from the Week 1 ungraded lab so you can compare outputs when just using stand-alone TFDV and when using it under TFX. Just to remind, the data can be used to predict if an individual earns more than or less than 50k US Dollars annually. Here is the description of the features again: 


* **age**: continuous.
* **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* **fnlwgt**: continuous.
* **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* **education-num**: continuous.
* **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* **race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* **sex**: Female, Male.
* **capital-gain**: continuous.
* **capital-loss**: continuous.
* **hours-per-week**: continuous.
* **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [3]:
# preview the first few rows of the CSV file
!head {_data_filepath}

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
39, State-gov, 77516, Bachelors, 13, Never-married, Adm-clerical, Not-in-family, White, Male, 2174, 0, 40, United-States, <=50K
50, Self-emp-not-inc, 83311, Bachelors, 13, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0, 0, 13, United-States, <=50K
38, Private, 215646, HS-grad, 9, Divorced, Handlers-cleaners, Not-in-family, White, Male, 0, 0, 40, United-States, <=50K
53, Private, 234721, 11th, 7, Married-civ-spouse, Handlers-cleaners, Husband, Black, Male, 0, 0, 40, United-States, <=50K
28, Private, 338409, Bachelors, 13, Married-civ-spouse, Prof-specialty, Wife, Black, Female, 0, 0, 40, Cuba, <=50K
37, Private, 284582, Masters, 14, Married-civ-spouse, Exec-managerial, Wife, White, Female, 0, 0, 40, United-States, <=50K
49, Private, 160187, 9th, 5, Married-spouse-absent, Other-service, Not-in-family, Black, Female, 0, 0,

### Crear el Contexto Interactivo

Cuando se empuja a la producción, se desea automatizar la ejecución de la tubería utilizando orquestadores como [Apache Beam](https://beam.apache.org/) y [Kubeflow](https://www.kubeflow.org/). No lo harás todavía y en su lugar ejecutarás el pipeline desde este cuaderno. Cuando experimentes en un entorno de cuaderno, estarás ejecutando *manualmente* los componentes del pipeline (es decir, tú eres el orquestador). Para ello, TFX proporciona el [Contexto Interactivo](https://github.com/tensorflow/tfx/blob/master/tfx/orchestration/experimental/interactive/interactive_context.py) para que puedas recorrer cada componente e inspeccionar sus salidas.
    
A continuación inicializarás el `Contexto Interactivo`. Esto creará una base de datos en el directorio `_pipeline_root` que los diferentes componentes utilizarán para guardar u obtener el estado de las ejecuciones de los componentes. Aprenderás más sobre esto en la semana 3 cuando hablemos de los metadatos ML. Por ahora, puedes pensar en él como el almacén de datos que hace posible que los diferentes componentes del pipeline trabajen juntos. 

*Nota: Puedes configurar la base de datos a la que conectarte, pero para este ejercicio utilizaremos la que viene por defecto, que es un archivo sqlite local recién creado.* ***Verás la advertencia después de ejecutar la celda de abajo y puedes ignorarla con seguridad.***

<details><summary>Texto Original</summary>


### Create the Interactive Context

When pushing to production, you want to automate the pipeline execution using orchestrators such as [Apache Beam](https://beam.apache.org/) and [Kubeflow](https://www.kubeflow.org/). You will not be doing that just yet and will instead execute the pipeline from this notebook. When experimenting in a notebook environment, you will be *manually* executing the pipeline components (i.e. you are the orchestrator). For that, TFX provides the [Interactive Context](https://github.com/tensorflow/tfx/blob/master/tfx/orchestration/experimental/interactive/interactive_context.py) so you can step through each component and inspect its outputs.
    
You will initialize the `InteractiveContext` below. This will create a database in the `_pipeline_root` directory which the different components will use to save or get the state of the component executions. You will learn more about this in Week 3 when we discuss ML Metadata. For now, you can think of it as the data store that makes it possible for the different pipeline components to work together. 

*Note: You can configure the database to connect to but for this exercise, we will just use the default which is a newly created local sqlite file.* ***You will see the warning after running the cell below and you can safely ignore it.***

In [4]:
# Inicializar el InteractiveContext con un archivo sqlite local.
# Si dejas `_pipeline_root` en blanco, entonces la db se creará en un directorio temporal.
# Puede ignorar la advertencia sobre la falta de un archivo de configuración.
context = InteractiveContext(pipeline_root=_pipeline_root)

## Ejecutar los componentes TFX de forma interactiva

Con esto, ahora puede ejecutar la tubería de forma interactiva. Verás cómo hacerlo a medida que vayas pasando por los diferentes componentes a continuación.
    
### ExampleGen

Iniciará el pipeline con el componente [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen). Esto lo hará:

* dividir los datos en conjuntos de entrenamiento y evaluación (por defecto: 2/3 de entrenamiento, 1/3 de evaluación).
* Convertirá cada fila de datos en formato `tf.train.Example`. Este [buffer de protocolo](https://developers.google.com/protocol-buffers) está diseñado para las operaciones de Tensorflow y es utilizado por los componentes TFX.
* comprimir y guardar la colección de datos bajo el directorio `_pipeline_root` para que otros componentes puedan acceder. Estos ejemplos se almacenan en formato `TFRecord`. Esto optimiza las operaciones de lectura y escritura dentro de Tensorflow especialmente si tienes una gran colección de datos.

Su constructor toma la ruta de tu fuente/directorio de datos. En nuestro caso, es la ruta `_data_root`. El componente soporta varias fuentes de datos como CSV, tf.Record, y BigQuery. Como nuestros datos son un archivo CSV, utilizaremos [CsvExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/CsvExampleGen) para ingerir los datos.

Ejecuta la celda de abajo para instanciar `CsvExampleGen`.

<details><summary>Texto Original</summary>

## Run TFX components interactively

With that, you can now run the pipeline interactively. You will see how to do that as you go through the different components below.
    
### ExampleGen

You will start the pipeline with the [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) component. This  will:

*   split the data into training and evaluation sets (by default: 2/3 train, 1/3 eval).
*   convert each data row into `tf.train.Example` format. This [protocol buffer](https://developers.google.com/protocol-buffers) is designed for Tensorflow operations and is used by the TFX components.
*   compress and save the data collection under the `_pipeline_root` directory for other components to access. These examples are stored in `TFRecord` format. This optimizes read and write operations within Tensorflow especially if you have a large collection of data.

Its constructor takes the path to your data source/directory. In our case, this is the `_data_root` path. The component supports several data sources such as CSV, tf.Record, and BigQuery. Since our data is a CSV file, we will use [CsvExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/CsvExampleGen) to ingest the data.

Run the cell below to instantiate `CsvExampleGen`.

In [5]:
# Instanciar ExampleGen con el conjunto de datos CSV de entrada
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

Puedes ejecutar el componente llamando al método `run()` del `InteractiveContext`.

<details><summary>Texto Original</summary>

You can execute the component by calling the `run()` method of the `InteractiveContext`.

In [6]:
# Executa ños componentes
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "./pipeline/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:3974460,xor_checksum:1635233824,sum_checksum:1635233824"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Observará que se muestra automáticamente una celda de salida con los resultados de la ejecución. Estos metadatos se registran en la base de datos creada anteriormente. Esto le permite hacer un seguimiento de las ejecuciones de su proyecto. Por ejemplo, si lo ejecuta de nuevo, notará que el `.execution_id` se incrementa.

La salida de los componentes se llama *artifacts* y puedes ver un ejemplo navegando por `.component.outputs > ['examples'] > Channel > ._artifacts > [0]` arriba. Muestra información como dónde se almacenan los datos convertidos (`.uri`) y las divisiones generadas (`.split_names`).

También puede examinar los artefactos de salida mediante programación con el código siguiente.

<details><summary>Texto Original</summary>

You will notice that an output cell showing the execution results is automatically shown. This metadata is recorded into the database created earlier. This allows you to keep track of your project runs. For example, if you run it again, you will notice the `.execution_id` incrementing.

The output of the components are called *artifacts* and you can see an example by navigating through  `.component.outputs > ['examples'] > Channel > ._artifacts > [0]` above. It shows information such as where the converted data is stored (`.uri`) and the splits generated (`.split_names`).

You can also examine the output artifacts programmatically with the code below.

In [7]:
# obtener el objeto artefacto
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/1


Si te lo preguntas, el `número` en `./pipeline/CsvExampleGen/examples/{number}` es el id de ejecución asociado a ese conjunto de datos. Si reinicia el núcleo de este espacio de trabajo y vuelve a ejecutar hasta esta celda, notará que se ha creado una nueva carpeta con un nombre de id diferente. Esto muestra que TFX está guardando versiones de sus datos para que pueda retroceder si quiere investigar una ejecución en particular.
    
Como se ha mencionado, los datos ingeridos se almacenan en el directorio mostrado en el campo `uri`. También se comprime usando `gzip` y puede verificarlo ejecutando la celda de abajo.

<details><summary>Texto Original</summary>

If you're wondering , the `number` in `./pipeline/CsvExampleGen/examples/{number}` is the execution id associated with that dataset. If you restart the kernel of this workspace and re-run up to this cell, you will notice a new folder with a different id name created. This shows that TFX is keeping versions of your data so you can roll back if you want to investigate a particular execution.
    
As mentioned, the ingested data is stored in the directory shown in the `uri` field. It is also compressed using `gzip` and you can verify by running the cell below.

In [8]:
# Obtener el URI del artefacto de salida que representa los ejemplos de entrenamiento
train_uri = os.path.join(artifact.uri, 'Split-train')

# Ver el contenido de la carpeta `train`.
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


En un entorno de cuaderno, puede ser útil examinar algunos ejemplos de los datos, especialmente si todavía está experimentando. Dado que la colección de datos se guarda en formato [TFRecord](https://www.tensorflow.org/tutorials/load_data/tfrecord), tendrá que utilizar métodos que funcionen con ese tipo de datos. Tendrás que desempaquetar los ejemplos individuales del archivo `TFRecord` y formatearlos para su impresión. Hagamos eso en las siguientes celdas:

<details><summary>Texto Original</summary>

In a notebook environment, it may be useful to examine a few examples of the data especially if you're still experimenting. Since the data collection is saved in [TFRecord format](https://www.tensorflow.org/tutorials/load_data/tfrecord), you will need to use methods that work with that data type. You will need to unpack the individual examples from the `TFRecord` file and format it for printing. Let's do that in the following cells:

In [9]:
# Obtener la lista de archivos de este directorio (todos los archivos TFRecord comprimidos)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Crear un `TFRecordDataset` para leer estos archivos
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [10]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extrae los registros del conjunto de datos dado.
    Args:
        dataset (TFRecordDataset): conjunto de datos guardado por ExampleGen
        num_records (int): número de registros a previsualizar
    '''
    
    # iniciar lista vacia
    records = []
    
    # Utilice el método `take()` para especificar el número de registros a obtener
    for tfrecord in dataset.take(num_records):
        
        # Obtener la propiedad numpy del tensor
        serialized_example = tfrecord.numpy()
        
        # Inicializar un `tf.train.Example()` para leer los datos serializados
        example = tf.train.Example()
        
        # Leer los datos del ejemplo (la salida es un mensaje de búfer de protocolo)
        example.ParseFromString(serialized_example)
        
        # convertir el mensaje del buffer de protocolo en un diccionario de Python
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [12]:
# Obtener 3 registros del conjunto de datos
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'age': {'int64List': {'value': ['39']}},
                           'capital-gain': {'int64List': {'value': ['2174']}},
                           'capital-loss': {'int64List': {'value': ['0']}},
                           'education': {'bytesList': {'value': ['IEJhY2hlbG9ycw==']}},
                           'education-num': {'int64List': {'value': ['13']}},
                           'fnlwgt': {'int64List': {'value': ['77516']}},
                           'hours-per-week': {'int64List': {'value': ['40']}},
                           'label': {'bytesList': {'value': ['IDw9NTBL']}},
                           'marital-status': {'bytesList': {'value': ['IE5ldmVyLW1hcnJpZWQ=']}},
                           'native-country': {'bytesList': {'value': ['IFVuaXRlZC1TdGF0ZXM=']}},
                           'occupation': {'bytesList': {'value': ['IEFkbS1jbGVyaWNhbA==']}},
                           'race': {'bytesList': {'value': ['IFdoaXRl']}},
                    

Ahora que `ExampleGen` ha terminado de ingerir los datos, el siguiente paso es el análisis de los mismos.
    
### StatisticsGen
El componente [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) calcula las estadísticas de su conjunto de datos para el análisis de los mismos, así como para su uso en los componentes posteriores (es decir, en los siguientes pasos del proceso). Como se ha mencionado anteriormente, este componente utiliza TFDV bajo el capó, por lo que su salida le resultará familiar.

`StatisticsGen` toma como entrada el conjunto de datos que acabamos de ingerir usando `CsvExampleGen`.

<details><summary>Texto Original</summary>

Now that `ExampleGen` has finished ingesting the data, the next step is data analysis.
    
### StatisticsGen
The [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) component computes statistics over your dataset for data analysis, as well as for use in downstream components (i.e. next steps in the pipeline). As mentioned earlier, this component uses TFDV under the hood so its output will be familiar to you.

`StatisticsGen` takes as input the dataset we just ingested using `CsvExampleGen`.

In [13]:
# Instanciar StatisticsGen con el conjunto de datos ingestado por ExampleGen
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

# Ejecutar el componente
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Puedes mostrar las estadísticas con el método `show()`.

*Nota: Puedes ignorar con seguridad la advertencia que se muestra al ejecutar la celda de abajo.*

<details><summary>Texto Original</summary>

You can display the statistics with the `show()` method.

*Note: You can safely ignore the warning shown when running the cell below.*

In [14]:
# Mostrar las estadísticas de salida
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

El componente [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) también utiliza TFDV para generar un esquema basado en sus estadísticas de datos. Como ha aprendido anteriormente, un esquema define los límites, tipos y propiedades esperados de las características de su conjunto de datos.

`SchemaGen` tomará como entrada las estadísticas que generamos con `StatisticsGen`, mirando la división de entrenamiento por defecto.

<details><summary>Texto Original</summary>

### SchemaGen

The [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) component also uses TFDV to generate a schema based on your data statistics. As you've learned previously, a schema defines the expected bounds, types, and properties of the features in your dataset.

`SchemaGen` will take as input the statistics that we generated with `StatisticsGen`, looking at the training split by default.

In [15]:
# instanciar SchemaGen con el conjunto de datos ingestado por StatisticsGen
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

A continuación, puede visualizar el esquema generado como una tabla.

<details><summary>Texto Original</summary>

You can then visualize the generated schema as a table.

In [16]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'education',STRING,required,,'education'
'label',STRING,required,,'label'
'marital-status',STRING,required,,'marital-status'
'native-country',STRING,required,,'native-country'
'occupation',STRING,required,,'occupation'
'race',STRING,required,,'race'
'relationship',STRING,required,,'relationship'
'sex',STRING,required,,'sex'
'workclass',STRING,required,,'workclass'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'label',"' <=50K', ' >50K'"
'marital-status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native-country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', ' Holand-Netherlands'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


Pasemos ahora al siguiente paso de la cadena y veamos si hay alguna anomalía en los datos.
    
### ExampleValidator

El componente [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) detecta anomalías en los datos basándose en el esquema generado en el paso anterior. Al igual que los dos componentes anteriores, también utiliza TFDV bajo el capó. 

El `ExampleValidator` tomará como entrada las estadísticas de `StatisticsGen` y el esquema de `SchemaGen`. Por defecto, compara las estadísticas de la división de evaluación con el esquema de la división de entrenamiento.

<details><summary>Texto Original</summary>

Let's now move to the next step in the pipeline and see if there are any anomalies in the data.
    
### ExampleValidator

The [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) component detects anomalies in your data based on the generated schema from the previous step. Like the previous two components, it also uses TFDV under the hood. 

`ExampleValidator` will take as input the statistics from `StatisticsGen` and the schema from `SchemaGen`. By default, it compares the statistics from the evaluation split to the schema from the training split.

In [17]:
# Instanciar ExampleValidator con los datos ingeridos de StatisticsGen y SchemaGen
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "./pipeline/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Al igual que con el componente anterior, también puede visualizar las anomalías en forma de tabla.

<details><summary>Texto Original</summary>

As with the previous component, you can also visualize the anomalies as a table.

In [19]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

Si no se detectan anomalías, puede pasar al siguiente paso del proceso.
    
### Transformación
El componente [Transform](https://www.tensorflow.org/tfx/guide/transform) realiza la ingeniería de características para los conjuntos de datos de entrenamiento y de servicio. Utiliza la biblioteca [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) introducida en el primer laboratorio no calificado de esta semana.

`Transform` tomará como entrada los datos de `ExampleGen`, el esquema de `SchemaGen`, así como un módulo que contiene la función de preprocesamiento.

En esta sección, se trabajará con un ejemplo de código de Transformación definido por el usuario. El pipeline necesita cargar esto como un módulo, por lo que es necesario utilizar el comando mágico `%% writefile` para guardar el archivo en el disco. Definamos primero algunas constantes que agrupen los atributos de los datos según las transformaciones que realizaremos más adelante. Este archivo también se guardará localmente.

<details><summary>Texto Original</summary>

With no anomalies detected, you can proceed to the next step in the pipeline.
    
### Transform
The [Transform](https://www.tensorflow.org/tfx/guide/transform) component performs feature engineering for both training and serving datasets. It uses the [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) library introduced in the first ungraded lab of this week.

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module containing the preprocessing function.

In this section, you will work on an example of a user-defined Transform code. The pipeline needs to load this as a module so you need to use the magic command `%% writefile` to save the file to disk. Let's first define a few constants that group the data's attributes according to the transforms we will perform later. This file will also be saved locally.

In [20]:
# Establecer las constantes del módulo nombre de archivo
_census_constants_module_file = 'census_constants.py'

In [21]:
%%writefile {_census_constants_module_file}

# Features with string data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = [
    'education', 'marital-status', 'occupation', 'race', 'relationship', 'workclass', 'sex', 'native-country'
]

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['age']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'age': 4}

# Feature that the model will predict
LABEL_KEY = 'label'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing census_constants.py


A continuación, trabajarás en el módulo que contiene `preprocessing_fn()`. Como has visto en el laboratorio anterior, esta función define cómo vas a transformar los datos en bruto en características sobre las que tu modelo puede entrenar (es decir, el siguiente paso en el pipeline). Utilizará las [funciones del módulo tft](https://www.tensorflow.org/tfx/transform/api_docs/python/tft) para realizar estas transformaciones. 

*Nota: Después de completar todo el cuaderno, te animamos a que vuelvas a esta sección y pruebes diferentes funciones tft aparte de las que ya se proporcionan a continuación. También puede modificar la agrupación de las claves de características en el archivo de constantes si lo desea. Por ejemplo, puede querer escalar algunas características a `[0, 1]` mientras que otras se escalan a la puntuación z. Esto será una buena práctica para la tarea de esta semana.

<details><summary>Texto Original</summary>

Next, you will work on the module that contains `preprocessing_fn()`. As you've seen in the previous lab, this function defines how you will transform the raw data into features that your model can train on (i.e. the next step in the pipeline). You will use the [tft module functions](https://www.tensorflow.org/tfx/transform/api_docs/python/tft) to make these transformations. 

*Note: After completing the entire notebook, we encourage you to go back to this section and try different tft functions aside from the ones already provided below. You can also modify the grouping of the feature keys in the constants file if you want. For example, you may want to scale some features to `[0, 1]` while others are scaled to the z-score. This will be good practice for this week's assignment.*

In [22]:
# Set the transform module filename
_census_transform_module_file = 'census_transform.py'

In [24]:
%%writefile {_census_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import census_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS     = census_constants.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = census_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS      = census_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT     = census_constants.FEATURE_BUCKET_COUNT
_LABEL_KEY                = census_constants.LABEL_KEY
_transformed_name         = census_constants.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])
    
    # Bucketize these features
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convert strings to indices in a vocabulary
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])

    return outputs

Overwriting census_transform.py


Ahora puede pasar los datos de entrenamiento, el esquema y el módulo de transformación al componente `Transform`. Puede ignorar los mensajes de advertencia generados por Apache Beam con respecto a las sugerencias de tipo.

<details><summary>Texto Original</summary>

You can now pass the training data, schema, and transform module to the `Transform` component. You can ignore the warning messages generated by Apache Beam regarding type hints.

In [25]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_census_transform_module_file))

# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 22
        uri: "./pipeline/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "./pipeline/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 23
        uri: "./pipeline/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "./pipeline/Transform/pre_transform_schema/5"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , a

Examinemos los artefactos de salida de `Transform` (es decir, `.component.outputs` de la celda de salida anterior). Este componente produce varias salidas:

* `transform_graph` es el gráfico que puede realizar las operaciones de preprocesamiento. Este gráfico se incluirá durante el entrenamiento y servirá para asegurar transformaciones consistentes de los datos entrantes.
* `transformed_examples` apunta a los datos de entrenamiento y evaluación preprocesados.
* `updated_analyzer_cache` son cálculos almacenados de ejecuciones anteriores.
    
    
Echa un vistazo al artefacto `transform_graph`.  Apunta a un directorio que contiene tres subdirectorios.

<details><summary>Texto Original</summary>

Let's examine the output artifacts of `Transform` (i.e. `.component.outputs` from the output cell above). This component produces several outputs:

* `transform_graph` is the graph that can perform the preprocessing operations. This graph will be included during training and serving to ensure consistent transformations of incoming data.
* `transformed_examples` points to the preprocessed training and evaluation data.
* `updated_analyzer_cache` are stored calculations from previous runs.
    
    
Take a peek at the `transform_graph` artifact.  It points to a directory containing three subdirectories.

In [27]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['metadata', 'transformed_metadata', 'transform_fn']

* El subdirectorio `metadata` contiene el esquema de los datos originales.
* El subdirectorio `transformed_metadata` contiene el esquema de los datos preprocesados. 
* El subdirectorio `transform_fn` contiene el gráfico de preprocesamiento real. 

También puedes echar un vistazo a los tres primeros ejemplos transformados utilizando la función de ayuda definida anteriormente.

<details><summary>Texto Original</summary>

* The `metadata` subdirectory contains the schema of the original data.
* The `transformed_metadata` subdirectory contains the schema of the preprocessed data. 
* The `transform_fn` subdirectory contains the actual preprocessing graph. 

You can also take a look at the first three transformed examples using the helper function defined earlier.

In [28]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [29]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'age_xf': {'int64List': {'value': ['2']}},
                           'capital-gain_xf': {'floatList': {'value': [0.021740217]}},
                           'capital-loss_xf': {'floatList': {'value': [0.0]}},
                           'education-num_xf': {'floatList': {'value': [0.8]}},
                           'education_xf': {'int64List': {'value': ['2']}},
                           'fnlwgt_xf': {'floatList': {'value': [0.044301897]}},
                           'hours-per-week_xf': {'floatList': {'value': [0.39795917]}},
                           'label_xf': {'int64List': {'value': ['0']}},
                           'marital-status_xf': {'int64List': {'value': ['1']}},
                           'native-country_xf': {'int64List': {'value': ['0']}},
                           'occupation_xf': {'int64List': {'value': ['3']}},
                           'race_xf': {'int64List': {'value': ['0']}},
                           'relationship_xf': {'int64List

**Felicitaciones!** Ahora ha ejecutado todos los componentes de nuestra línea de producción. Usted obtendrá la práctica también con la formación y la evaluación del modelo en los próximos cursos, pero por ahora, le animamos a tratar de explorar los diferentes componentes que acabamos de discutir. Como se mencionó anteriormente, un ejercicio útil para la próxima asignación es familiarizarse con el uso de diferentes funciones `tft` en su módulo de transformación. Intente explorar la [documentación](https://www.tensorflow.org/tfx/transform/api_docs/python/tft) y vea qué otras funciones puede utilizar en el módulo de transformación. También puedes hacer el reto opcional que aparece a continuación para practicar más.

**Desafío opcional:** Utilizando este cuaderno como referencia, cargue el [Conjunto de datos de demanda de bicicletas compartidas de Seúl](https://archive.ics.uci.edu/ml/datasets/Seoul+Bike+Sharing+Demand) y ejecútelo a través de las cinco etapas de la cadena de producción discutida aquí. Primero pasarás por los componentes de ingestión y validación de datos y, finalmente, estudiarás las características del conjunto de datos y lo transformarás a un formato que pueda consumir un modelo. Una vez que hayas terminado, puedes visitar este [tema de Discourse](https://community.deeplearning.ai/t/bike-sharing-dataset-in-c2-w2-lab-2-feature-engineering-pipeline/38979) donde uno de tus mentores, Fabio, ha compartido su solución. ¡Siéntase libre de discutir y compartir su solución también!

<details><summary>Texto Original</summary>

**Congratulations!** You have now executed all the components in our pipeline. You will get hands-on practice as well with training and model evaluation in future courses but for now, we encourage you to try exploring the different components we just discussed. As mentioned earlier, a useful exercise for the upcoming assignment is to be familiar with using different `tft` functions in your transform module. Try exploring the [documentation](https://www.tensorflow.org/tfx/transform/api_docs/python/tft) and see what other functions you can use in the transform module. You can also do the optional challenge below for more practice.

**Optional Challenge:** Using this notebook as reference, load the [Seoul Bike Sharing Demand Dataset](https://archive.ics.uci.edu/ml/datasets/Seoul+Bike+Sharing+Demand) and run it through the five stages of the pipeline discussed here. You will first go through the data ingestion and validation components then finally, you will study the dataset's features and transform it to a format that a model can consume. Once you're done, you can visit this [Discourse topic](https://community.deeplearning.ai/t/bike-sharing-dataset-in-c2-w2-lab-2-feature-engineering-pipeline/38979) where one of your mentors, Fabio, has shared his solution. Feel free to discuss and share your solution as well!